### Задание

- скачать набор данных маркетинговых кампаний https://www.kaggle.com/davinwijaya/customer-retention
- поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.
- сделать разбиение набора данных не тренировочную и тестовую выборки
- сделать feature engineering на ваше усмотрение (допускается свобода выбора методов)
- провести uplift-моделирование 3 способами: одна модель с признаком коммуникации (S learner), модель с трансформацией таргета (трансформация классов п. 2. 1) и вариант с двумя независимыми моделями
- в конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% этих 3 моделей
- построить модель UpliftTreeClassifier и попытаться описать словами полученное дерево
- (опционально) для модели S learner (модель с дополнительным признаком коммуникации) построить зависимость таргета (конверсии - поле conversion) от значения uplift: 1) сделать прогноз и получить uplift для тестовой выборки 2) отсортировать тестовую выборку по uplift по убыванию 3) разбить на децили (pandas qcut вам в помощь) 4) для каждого дециля посчитать среднюю conversion
- (опционально) построить модель UpliftRandomForestClassifier и попытаться описать словами полученное дерево

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel, ClassTransformation, TwoModels

from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('./HW_data.csv')
df.rename(columns={'offer': 'treatment', 'conversion': 'target'}, inplace=True)
df['treatment'] = df['treatment'].apply(lambda x: 0 if x == 'No Offer' else 1)
df.head(3)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,1,0
1,6,329.08,1,1,Rural,1,Web,0,0
2,7,180.65,0,1,Surburban,1,Web,1,0


In [3]:
X = df[df.columns.difference(['target'], sort=False)]
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

treat_train, treat_test = X_train['treatment'], X_test['treatment']

X_train = X_train.copy().drop('treatment', axis=1)
X_test = X_test.copy().drop('treatment', axis=1)

cat_features = ['zip_code', 'channel']

In [4]:
models_results = {
    'approach': [],
    'uplift@10%': [],
    'uplift@20%': []
}

#### Модель с признаком коммуникации

In [5]:
sm = SoloModel(CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True))
sm = sm.fit(X_train, y_train, treat_train, estimator_fit_params={'cat_features': cat_features})

uplift_sm = sm.predict(X_test)

for k in [0.1, 0.2]:
    sm_score = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=treat_test, strategy='by_group', k=k)
    models_results[f'uplift@{int(k * 100)}%'].append(sm_score)

models_results['approach'].append('SoloModel')

#### Модель с трансформацией классов

In [6]:
ct = ClassTransformation(CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True))
ct = ct.fit(X_train, y_train, treat_train, estimator_fit_params={'cat_features': cat_features})

uplift_ct = ct.predict(X_test)

for k in [0.1, 0.2]:
    ct_score = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=treat_test, strategy='by_group', k=k)
    models_results[f'uplift@{int(k * 100)}%'].append(ct_score)

models_results['approach'].append('ClassTransformation')

#### Метод с двумя моделями

In [7]:
tm = TwoModels(
    estimator_trmnt=CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True), 
    estimator_ctrl=CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True), 
    method='vanilla')

tm = tm.fit(
    X_train, y_train, treat_train,
    estimator_trmnt_fit_params={'cat_features': cat_features}, 
    estimator_ctrl_fit_params={'cat_features': cat_features}
)

uplift_tm = tm.predict(X_test)

for k in [0.1, 0.2]:
    tm_score = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=treat_test, strategy='by_group', k=k)
    models_results[f'uplift@{int(k * 100)}%'].append(tm_score)
    
models_results['approach'].append('TwoModels')

#### Вывод результатов

In [8]:
pd.DataFrame(models_results)

,approach,uplift@10%,uplift@20%
0,SoloModel,0.108870,0.106608
1,ClassTransformation,0.115533,0.098877
2,TwoModels,0.089004,0.096113
